In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8973,2024-06-09,México Cibacopa,00:00,Zonkeys de Tijuana,Rayos de Hermosillo,1.35,3.02,171.5,1.80,1.86,0.0,0.00,0.00,lOHUAEzP,0.740741,0.331126,0.555556,0.537634,0.071867,134.166,40.089682,0.298807,2.4,1.341641,0.559017,152.60,1.366,0.125817,0.092106,31.0,132.192,24.586475,0.185991,1.2,1.643168,1.369306,115.92,1.504,0.178969,0.118996,-6.0,109,84,1.40,1.38,116.042,158.374,0.540443,0.023184,NaN,-1.28,-0.256,-1.367188,0.797541,0.8,0.002459,0.36,0.072,28.055556,0.534188,0.4,-0.134188
8974,2024-06-09,Indonésia Ibl,10:00,Bumi Borneo,Hangtuah,1.58,2.29,163.5,1.83,1.83,0.0,0.00,0.00,6kBfEAAb,0.632911,0.436681,0.546448,0.546448,0.069593,340.338,229.554839,0.674491,0.6,1.341641,2.236068,98.40,4.184,3.014105,0.720388,-39.0,404.544,258.452462,0.638874,0.6,1.341641,2.236068,137.76,4.758,2.757076,0.579461,-72.0,82,82,1.20,1.68,260.238,424.728,0.259455,0.000000,NaN,-2.10,-0.420,-1.380952,0.000000,0.0,0.000000,-3.83,-0.766,-1.684073,0.405869,0.3,-0.105869
8975,2024-06-09,Itália Série A2,13:00,Agrigento,Chiusi,1.91,1.81,151.5,1.88,1.79,5.5,1.40,1.42,EV8DqJ3B,0.523560,0.552486,0.531915,0.558659,0.076046,151.982,38.908350,0.256006,1.8,1.643168,0.912871,142.00,1.996,0.775132,0.388343,6.0,116.292,25.464645,0.218972,2.4,1.341641,0.559017,140.08,1.392,0.183494,0.131820,68.0,100,103,1.42,1.36,168.702,160.136,0.038016,0.034681,0.010030,2.86,0.572,1.590909,0.517811,0.5,-0.017811,3.07,0.614,1.319218,0.351257,0.6,0.248743
8976,2024-06-09,Itália Série A2,13:00,Cento,Nardo,1.42,2.72,152.5,1.83,1.87,0.0,0.00,0.00,Ac2MscYN,0.704225,0.367647,0.546448,0.534759,0.071872,121.286,36.366607,0.299842,1.8,1.643168,0.912871,184.14,1.690,0.719757,0.425892,-15.0,146.732,49.019914,0.334078,1.8,1.643168,0.912871,140.25,1.906,0.652940,0.342571,4.0,62,85,2.97,1.65,101.310,0.000,0.444077,0.015289,NaN,0.38,0.076,5.526316,0.712507,0.6,-0.112507,0.00,0.000,inf,0.000000,0.0,0.000000
8977,2024-06-09,Itália Série A2,16:00,Fortitudo Bologna,Trapani,2.29,1.58,147.5,1.87,1.80,0.0,0.00,0.00,KACx3lU6,0.436681,0.632911,0.534759,0.555556,0.069593,169.718,79.861731,0.470555,1.8,1.643168,0.912871,144.64,2.462,1.074486,0.436428,9.0,105.326,20.029159,0.190163,2.4,1.341641,0.559017,93.81,1.360,0.158114,0.116260,33.0,64,59,2.26,1.59,113.944,109.746,0.259455,0.026974,NaN,2.63,0.526,2.452471,0.728642,0.8,0.071358,-2.20,-0.440,-1.318182,0.739816,0.6,-0.139816
8978,2024-06-09,Lituânia Lkl,11:20,BC Wolves,Lietkabelis,1.56,2.36,168.5,1.86,1.87,0.0,0.00,0.00,6yOfDjif,0.641026,0.423729,0.537634,0.534759,0.064754,224.364,59.460512,0.265018,1.2,1.643168,1.369306,193.59,2.628,0.633577,0.241087,3.0,317.806,162.680041,0.511885,0.6,1.341641,2.236068,113.15,4.012,2.155834,0.537346,-37.0,81,73,2.39,1.55,205.050,298.478,0.288615,0.003791,NaN,2.41,0.482,1.161826,0.667657,0.8,0.132343,1.24,0.248,5.483871,0.446049,0.5,0.053951
8979,2024-06-09,México Cibacopa,20:15,Astros,Angeles CDMX,1.26,3.55,192.5,1.82,1.88,0.0,0.00,0.00,YTWT65Zj,0.793651,0.281690,0.549451,0.531915,0.075341,133.524,14.381854,0.107710,2.4,1.341641,0.559017,136.08,1.258,0.089275,0.070966,75.0,201.820,57.982444,0.287298,1.8,1.643168,0.912871,285.19,2.134,0.896370,0.420042,-1.0,108,79,1.26,3.61,128.880,228.156,0.673295,0.022933,NaN,-0.01,-0.002,-130.000000,0.826389,0.9,0.073611,-0.69,-0.138,-18.478261,0.495385,0.5,0.004615
8980,2024-06-09,C

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8987,08:00,Indonésia Ibl,RANS PIK,Kesatria Bengawan Solo,2.53,Back Home
8991,07:15,Filipinas Copa Das Filipinas,San Miguel Beermen,Meralco Bolts,1.56,Back Home
9005,17:00,Eua Wnba,Dallas Wings F,Phoenix Mercury F,1.69,Back Home
9006,20:00,Eua Wnba,Minnesota Lynx F,Seattle Storm F,1.65,Back Home
